# Tarea SQL


El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.  

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

# Objetivo

Analizar la base de datos para comprender el mercado de los amantes de los libros y crear un producto o servicio que sea destacable al plantear las necesidades y preferencias identificadas a partir de los datos recopilados.

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine, inspect
# import os
# from dotenv import load_dotenv
# load_dotenv()

In [2]:
# importar librerías

db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}           # nombre de la base de datos

connection_string = f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# función para ejecutar una consulta SQL utilizando pandas
def query_pandas(query):
    df= pd.io.sql.read_sql(query, con = engine)

    return df

In [4]:
# tabla books
query = '''SELECT * 
           FROM 
           public.books
           LIMIT 5
           '''

In [5]:
# primeras 5 filas de tabla books
query_pandas(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [6]:
# tabla authors
query = '''SELECT * 
           FROM 
           public.authors
           LIMIT 5
           '''

# primeras 5 filas de tabla 
query_pandas(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
# tabla publishers
query = '''SELECT * 
           FROM 
           public.publishers
           LIMIT 5
           '''

# primeras 5 filas de tabla 
query_pandas(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
# tabla ratings
query = '''SELECT * 
           FROM 
           public.ratings
           LIMIT 5
           '''

# primeras 5 filas de tabla 
query_pandas(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
# tabla reviews
query = '''SELECT * 
           FROM 
           public.reviews
           LIMIT 5
           '''

# primeras 5 filas de tabla 
query_pandas(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Número de libros publicados después del 1 de enero de 2000.

In [10]:
# query
query = '''SELECT 
               COUNT(title) AS total_libros
           FROM 
                public.books
           WHERE publication_date > '2000-01-01'
           '''

# primeras 5 filas de tabla 
libros_totales= query_pandas(query)
libros_totales

,total_libros
0,819


# Número de reseñas de usuarios y la calificación promedio para cada libro

In [11]:
# tabla reviews
query = '''SELECT 
               bk.book_id,
               bk.title,
               COUNT(r.review_id) AS num_reviews,
               AVG(rt.rating) AS avg_rating
           FROM 
                public.books AS bk
            INNER JOIN
                public.reviews AS r ON bk.book_id = r.book_id
            INNER JOIN 
                public.ratings AS rt ON bk.book_id = rt.book_id
           GROUP BY 
                bk.book_id,
                bk.title
            ORDER BY
                num_reviews DESC
           '''

# primeras 5 filas de tabla 
reviews_ratings_books = query_pandas(query)
reviews_ratings_books.head()

,book_id,title,num_reviews,avg_rating
0,948,Twilight (Twilight #1),1120,3.662500
1,750,The Hobbit or There and Back Again,528,4.125000
2,673,The Catcher in the Rye,516,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry...,480,4.287500


# Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis)

In [12]:
query = '''SELECT 
               p.publisher_id,
               p.publisher,
               COUNT(bk.book_id) AS total_books
           FROM 
                public.books AS bk
            INNER JOIN
                public.publishers AS p ON bk.publisher_id = p.publisher_id
           WHERE
               bk.num_pages > 50
           GROUP BY 
               p.publisher_id,
               p.publisher
            ORDER BY
                total_books DESC
           '''

# primeras 5 filas de tabla 
publishers_books_50_pages = query_pandas(query)
publishers_books_50_pages.head()

,publisher_id,publisher,total_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19


# Identificación del autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones

In [13]:
query = '''SELECT
               a.author_id,
               a.author,
               AVG(rt.rating) AS avg_rating
           FROM 
                public.books AS bk
            INNER JOIN
                public.authors AS a ON bk.author_id = a.author_id
            INNER JOIN
                public.ratings AS rt ON bk.book_id = rt.book_id
           GROUP BY
               a.author_id, 
               a.author
            HAVING
               COUNT(rt.rating_id) >= 50
            ORDER BY
                avg_rating DESC
           '''

# primeras 5 filas de tabla 
authors_ratings = query_pandas(query)
authors_ratings.head()

,author_id,author,avg_rating
0,130,Diana Gabaldon,4.300000
1,236,J.K. Rowling/Mary GrandPré,4.288462
2,3,Agatha Christie,4.283019
3,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
4,240,J.R.R. Tolkien,4.240964


# Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [42]:
query = ''' 
SELECT 
    AVG(num_reviews) AS avg_text_reviews_per_user
FROM (
    SELECT 
        username, 
        COUNT(text) AS num_reviews
    FROM 
        public.reviews
    WHERE 
        username IN (
            SELECT 
                username
            FROM 
                public.ratings
            GROUP BY 
                username
            HAVING 
                COUNT(rating_id) > 50
                )
GROUP BY username
    ) AS avg_reviews_per_user
        ''' 

# primeras 5 filas de tabla 
users_reviews_mean = query_pandas(query)
users_reviews_mean.head()

,avg_text_reviews_per_user
0,24.333333


# Conclusiones

<div style="background-color: lightyellow; padding: 10px;">

<span style="color: darkblue;">  
    
Después del 01 de enero del 2000 se publicaron un total de 819 libros.  

El top 5 de losl libros con más reseñas son: Twilight #1, The Hobbit or There and Back Again, The Catcher in the Rye, Harry Potter and the Prisoner of Azkaban y Harry Potter and the Chamber of Secrets. Twilight #1(1120), tuvo muchas reseñas, pero el rpomedio de las calificaciones fue de 3.7, mientras que, Harry Potter and the Prisoner of Azkaban tuvo menos reseñas (492), pero su calificación fue la más alta, 4.4.  

El top 5 de las editoriales que publicaron libros con más de 50 páginas fueron: Penguin Books, Vintage, Grand Central Publishing, Penguin Classics y Ballantine Books.     

Los 5 autores con las mayores calificaciones de sus libros fueron: Diana Gabaldon, J.K. Rowling/Mary GrandPré, Agatha Christie, Markus Zusak/Cao Xuân Việt Khương y J.R.R. Tolkien.  

Los usuarios y usuarias que calificaron más de 50 libros, en promedio hicieron 166.7 reseñas de texto.  


</span>
    
</div>